<a href="https://colab.research.google.com/github/MatthewYancey/GANime/blob/master/src/train_lama_default.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tain LaMa
This notebok runs the default training for LaMa

In [ ]:
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

In [ ]:
import os
import shutil
import glob

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# train_zip = '/content/gdrive/MyDrive/repos/GANime/data_out/pokemon/train_png.zip'
val_zip = '/content/gdrive/MyDrive/repos/GANime/data_out/pokemon/test_masks.zip'
eval_zip = '/content/gdrive/MyDrive/repos/GANime/data_out/pokemon/test_masks.zip'
dataset_dir = '/content/lama/my_dataset/'

In [ ]:
print('\n> Cloning the repo')
!git clone https://github.com/saic-mdal/lama.git

print('\n> Install dependencies')
!pip install -r lama/requirements.txt --quiet
!pip install wget --quiet

print('\n> Changing the dir to:')
% cd /content/lama

# downloads the models for evaluations
print('Downloading resnet model for evaluation')
!mkdir -p ade20k/ade20k-resnet50dilated-ppm_deepsup/
!wget -P ade20k/ade20k-resnet50dilated-ppm_deepsup/ http://sceneparsing.csail.mit.edu/model/pytorch/ade20k-resnet50dilated-ppm_deepsup/encoder_epoch_20.pth

# see if you don't need this
# print('\n> Download the model')
# !curl -L $(yadisk-direct https://disk.yandex.ru/d/EgqaSnLohjuzAg) -o lama-models.zip
# !unzip lama-models.zip

print('>fixing opencv')
!pip uninstall opencv-python-headless -y --quiet
!pip install opencv-python-headless==4.1.2.30 --quiet

print('\n> Init mask-drawing code')
import base64, os
from IPython.display import HTML, Image
from google.colab.output import eval_js
from base64 import b64decode
import matplotlib.pyplot as plt
import numpy as np
import wget
from shutil import copyfile
import shutil

In [ ]:
# val_source # 2000 or more images
# visual_test_source # 100 or more images
# eval_source # 2000 or more images

# copy the training set over
try:
    os.mkdir(dataset_dir)
except FileExistsError:
    pass
shutil.copy(val_zip, dataset_dir + 'val.zip')
shutil.unpack_archive(dataset_dir + 'val.zip', dataset_dir + '/val/')

# makes a copy for eval and visual test
shutil.copytree(dataset_dir + 'val/', dataset_dir + 'eval/')
shutil.copytree(dataset_dir + 'val/', dataset_dir + 'visual_test/')

In [ ]:
# copies images to the training folder
img_list = glob.glob(dataset_dir + '/val/*')
print(f'Number of images: {len(img_list)}')
img_list = [f for f in img_list if 'mask' not in f]
print(f'Number of images: {len(img_list)}')

In [ ]:
glob.glob(dataset_dir + '/train/*')[0]

In [ ]:
# picks 130k images and resizes them
if os.path.isdir(dataset_dir + 'train/'):
    shutil.rmtree(dataset_dir + 'train/')
os.mkdir(dataset_dir + 'train/')

import random
import cv2

for i in range(130000):
    img_file = random.choice(img_list)
    img = cv2.imread(img_file)
    cv2.imwrite(dataset_dir + 'train/' + os.path.basename(img_file).replace('.png', '.jpg'), img)
    
    if i % 10000 == 0:
        print(i)

In [ ]:
# makes the config file for training
!touch my_dataset.yaml
!echo "data_root_dir: $(pwd)/my_dataset/" >> my_dataset.yaml
!echo "out_root_dir: $(pwd)/experiments/" >> my_dataset.yaml
!echo "tb_dir: $(pwd)/tb_logs/" >> my_dataset.yaml
!mv my_dataset.yaml ${PWD}/configs/training/location/

# change /content/lama/configs/training/trainer/any_gpu_large_ssim_ddp_final.yaml limit_train_batches: 2600 # was 25000

In [ ]:
!export TORCH_HOME=$(pwd) && export USER='matt' && export PYTHONPATH=. && python3 bin/train.py -cn lama-fourier location=my_dataset data.batch_size=10
# !cat ${PWD}/configs/training/data/abl-04-256-mh-dist